## Modelo de programación lineal entera en "Sagemath" para la solución del problema de asignación de ingenieros calificados a solicitudes de servicio en una empresa de software.

<br/>

Carga de las librerías auxiliares para el manejo de las estructuras de datos en Python y Sagemath. Este modelo se ejecuta utilizando un kernel de la herramienta Sagemath, que debe ser cargado previamente a la ejecución de las instrucciones siguientes.

In [1]:
import pandas as pd

Declaración de las estructuras de datos necesarias para la especificación de la función objetivo y las restricciones.

A continuación se presenta la especificación de la matriz de solicitudes de servicio.  En esta matriz cada cliente selecciona entre diferentes opciones de tipos de servicio, en este caso se han etiquetado con números del 1 al 6.
De tal forma que el cliente 1 ha solicitado 2 servicios del tipo 1, el cliente 2 solicita un servicio del tipo 3, etc.

In [2]:
data = [['C1', 2, 0, 0, 0, 0, 0], 
        ['C2', 0, 0, 1, 0, 0, 0],
        ['C3', 1, 1, 0, 0, 0, 0],
        ['C4', 0, 0, 0, 0, 2, 0],
        ['C5', 0, 0, 0, 1, 0, 1]]

solicitudes = pd.DataFrame(data, columns=['IdCliente', 'Servicio01', 'Servicio02', 
                                    'Servicio03', 'Servicio04', 'Servicio05', 
                                    'Servicio06'])

solicitudes = solicitudes.set_index(['IdCliente'])
solicitudes

,Servicio01,Servicio02,Servicio03,Servicio04,Servicio05,Servicio06
IdCliente,,,,,,
C1,2,0,0,0,0,0
C2,0,0,1,0,0,0
C3,1,1,0,0,0,0
C4,0,0,0,0,2,0
C5,0,0,0,1,0,1


La siguiente matriz es la de acreditaciones, que se refiere a las destrezas que cada ingeniero ha desarrollado, que en conjunto con su experiencia le permiten llevar a cabo ciertos tipo de solicitudes de servicio.
Para este caso, el ingeniero 1, puede resolver solicitudes de servicio del tipo 3, 4 y 6, de manera similar los otros ingenieros pueden ejecutar otros servicios.

In [3]:
data = [['I1', 0, 0, 1, 1, 0, 1], 
        ['I2', 0, 0, 1, 0, 1, 1],
        ['I3', 1, 1, 0, 1, 0, 0],
        ['I4', 0, 1, 0, 0, 1, 1],
        ['I5', 0, 0, 0, 1, 0, 1],
        ['I6', 1, 1, 1, 1, 0, 0]]

acreditaciones = pd.DataFrame(data, columns=['IdIngeniero', 'Servicio01', 'Servicio02', 
                                    'Servicio03', 'Servicio04', 'Servicio05', 
                                    'Servicio06'])

acreditaciones = acreditaciones.set_index(['IdIngeniero'])
acreditaciones

,Servicio01,Servicio02,Servicio03,Servicio04,Servicio05,Servicio06
IdIngeniero,,,,,,
I1,0,0,1,1,0,1
I2,0,0,1,0,1,1
I3,1,1,0,1,0,0
I4,0,1,0,0,1,1
I5,0,0,0,1,0,1
I6,1,1,1,1,0,0


## === Eliminar porque no se incluyen en el modelo!!! ===

In [4]:
data = [['I1', 2], 
        ['I2', 1],
        ['I3', 3],
        ['I4', 2],
        ['I5', 1],
        ['I6', 3]]

libres = pd.DataFrame(data, columns=['IdIngeniero', 'SemanasLibres'])

libres = libres.set_index(['IdIngeniero'])
libres

,SemanasLibres
IdIngeniero,
I1,2
I2,1
I3,3
I4,2
I5,1
I6,3


<br/>

En la matriz siguiente podemos observar los costos totales que se pueden incurrir si un ingeniero cualquiera, ejecuta el servicio que el cliente $x$ está solicitando. 

In [5]:
data = [['I1', 120, 113,  91,  97,  60], 
        ['I2',  68,  97,  68, 127, 113],
        ['I3', 111, 115, 120, 114, 119],
        ['I4',  93, 113, 113,  93, 112],
        ['I5', 108,  67, 115, 115, 114],
        ['I6', 125, 107,  60, 113,  64]]

costos = pd.DataFrame(data, columns=['IdIngeniero', 'C1', 'C2', 'C3', 'C4', 'C5'])

costos = costos.set_index(['IdIngeniero'])
costos

,C1,C2,C3,C4,C5
IdIngeniero,,,,,
I1,120,113,91,97,60
I2,68,97,68,127,113
I3,111,115,120,114,119
I4,93,113,113,93,112
I5,108,67,115,115,114
I6,125,107,60,113,64


<br/>

Las siguientes lineas de código, llevan a cabo la definición del modelo de programación lineal.  Se indica que es un modelo de minimización (de costos).  Y declaramos una nueva variable llamada `Asignación` que es no negativa y entera.

In [6]:
programa = MixedIntegerLinearProgram(maximization=False)
asignacion = programa.new_variable(integer=True, nonnegative = True, name='Asignación')

<br/>

Las lineas de código siguientes, se utilizan para hacer un poco mas entendible las solicitudes de servicio de los clientes y su asignación a un conjunto posible de ingenieros calificados.

También en estas lineas de código se arma la función objetivo en la variable "suma", que contiene la sumatoria de costos de asignar un ingeniero $i$ a un cliente $j$.
En el impreso se muestra la función objetivo completa.  La función objetivo solo necesita de 16 variables ($x_0$ a $x_{15}$) para su definición, ya que esas son las combinaciones suficientes de clientes, ingenieros y solicitudes de servicio.

In [7]:
suma = 0
for i in solicitudes.index:
    if solicitudes.Servicio01.loc[i] > 0:
        print("El cliente", i, "requiere", solicitudes.Servicio01.loc[i], "servicios. Los ingenieros posibles son:")
        for j in acreditaciones.index:
            if acreditaciones.Servicio01.loc[j] > 0:
                print("\tEl ingeniero" , j, "con costo de $", costos[i].loc[j])
                suma = suma + costos[i].loc[j] * asignacion[i, j]
    if solicitudes.Servicio02.loc[i] > 0:
        print("El cliente", i, "requiere", solicitudes.Servicio02.loc[i], "servicios. Los ingenieros posibles son:")
        for j in acreditaciones.index:
            if acreditaciones.Servicio02.loc[j] > 0:
                print("\tEl ingeniero" , j, "con costo de $", costos[i].loc[j])
                suma = suma + costos[i].loc[j] * asignacion[i, j]
    if solicitudes.Servicio03.loc[i] > 0:
        print("El cliente", i, "requiere", solicitudes.Servicio03.loc[i], "servicios. Los ingenieros posibles son:")
        for j in acreditaciones.index:
            if acreditaciones.Servicio03.loc[j] > 0:
                print("\tEl ingeniero" , j, "con costo de $", costos[i].loc[j])
                suma = suma + costos[i].loc[j] * asignacion[i, j]
    if solicitudes.Servicio04.loc[i] > 0:
        print("El cliente", i, "requiere", solicitudes.Servicio04.loc[i], "servicios. Los ingenieros posibles son:")
        for j in acreditaciones.index:
            if acreditaciones.Servicio04.loc[j] > 0:
                print("\tEl ingeniero" , j, "con costo de $", costos[i].loc[j])
                suma = suma + costos[i].loc[j] * asignacion[i, j]
    if solicitudes.Servicio05.loc[i] > 0:
        print("El cliente", i, "requiere", solicitudes.Servicio05.loc[i], "servicios. Los ingenieros posibles son:")
        for j in acreditaciones.index:
            if acreditaciones.Servicio05.loc[j] > 0:
                print("\tEl ingeniero" , j, "con costo de $", costos[i].loc[j])
                suma = suma + costos[i].loc[j] * asignacion[i, j]
    if solicitudes.Servicio06.loc[i] > 0:
        print("El cliente", i, "requiere", solicitudes.Servicio06.loc[i], "servicios. Los ingenieros posibles son:")
        for j in acreditaciones.index:
            if acreditaciones.Servicio06.loc[j] > 0:
                print("\tEl ingeniero" , j, "con costo de $", costos[i].loc[j])
                suma = suma + costos[i].loc[j] * asignacion[i, j]

suma = suma - (costos['C3'].loc['I3']*asignacion['C3','I3'] + costos['C3'].loc['I6']*asignacion['C3','I6'] + costos['C5'].loc['I1']*asignacion['C5','I1'] + costos['C5'].loc['I5']*asignacion['C5','I5'])

programa.set_objective(suma)
programa.show()

El cliente C1 requiere 2 servicios. Los ingenieros posibles son:
	El ingeniero I3 con costo de $ 111
	El ingeniero I6 con costo de $ 125
El cliente C2 requiere 1 servicios. Los ingenieros posibles son:
	El ingeniero I1 con costo de $ 113
	El ingeniero I2 con costo de $ 97
	El ingeniero I6 con costo de $ 107
El cliente C3 requiere 1 servicios. Los ingenieros posibles son:
	El ingeniero I3 con costo de $ 120
	El ingeniero I6 con costo de $ 60
El cliente C3 requiere 1 servicios. Los ingenieros posibles son:
	El ingeniero I3 con costo de $ 120
	El ingeniero I4 con costo de $ 113
	El ingeniero I6 con costo de $ 60
El cliente C4 requiere 2 servicios. Los ingenieros posibles son:
	El ingeniero I2 con costo de $ 127
	El ingeniero I4 con costo de $ 93
El cliente C5 requiere 1 servicios. Los ingenieros posibles son:
	El ingeniero I1 con costo de $ 60
	El ingeniero I3 con costo de $ 119
	El ingeniero I5 con costo de $ 114
	El ingeniero I6 con costo de $ 64
El cliente C5 requiere 1 servicios. Los 

<br/>

El modelo de programación lineal, requiere de la definición de restricciones que limitan la minimización de la función objetivo.
En nuestro problema se tienen dos tipos de restricciones, la primera relacionada con que cada ingeniero debe resolver máximo, la cantidad de solicitudes de cada cliente y también resolver solo aquellas para las que está calificado.

In [8]:
programa.add_constraint(asignacion[('C2', 'I1')] + asignacion[('C5', 'I1')] <= 2)
programa.add_constraint(asignacion[('C2', 'I2')] + asignacion[('C4', 'I2')] + asignacion[('C5', 'I2')] <= 1)
programa.add_constraint(asignacion[('C1', 'I3')] + asignacion[('C3', 'I3')] + asignacion[('C5', 'I3')] <= 3)
programa.add_constraint(asignacion[('C3', 'I4')] + asignacion[('C4', 'I4')] + asignacion[('C5', 'I4')] <= 2)
programa.add_constraint(asignacion[('C5', 'I5')] <= 1)
programa.add_constraint(asignacion[('C1', 'I6')] + asignacion[('C2', 'I6')] + asignacion[('C3', 'I6')] + asignacion[('C5', 'I6')] <= 3)

<br/>

En estas restricciones se maneja que cada cliente reciba a un ingeniero para resolver su solicitud de servicio y no más de lo solicitado.

In [9]:
programa.add_constraint(asignacion[('C1', 'I3')] + asignacion[('C1', 'I6')] == 2)
programa.add_constraint(asignacion[('C2', 'I1')] + asignacion[('C2', 'I2')] + asignacion[('C2', 'I6')] == 1)
programa.add_constraint(asignacion[('C3', 'I3')] + asignacion[('C3', 'I6')] + asignacion[('C3', 'I4')] == 2)
programa.add_constraint(asignacion[('C4', 'I2')] + asignacion[('C4', 'I4')] == 2)
programa.add_constraint(asignacion[('C5', 'I1')] + asignacion[('C5', 'I3')] + asignacion[('C5', 'I5')] + asignacion[('C5', 'I6')] + asignacion[('C5', 'I2')] + asignacion[('C5', 'I4')] == 2)
programa.show()

Minimization:
  111.0 Asignación[('C1', 'I3')] + 125.0 Asignación[('C1', 'I6')] + 113.0 Asignación[('C2', 'I1')] + 97.0 Asignación[('C2', 'I2')] + 107.0 Asignación[('C2', 'I6')] + 120.0 Asignación[('C3', 'I3')] + 60.0 Asignación[('C3', 'I6')] + 113.0 Asignación[('C3', 'I4')] + 127.0 Asignación[('C4', 'I2')] + 93.0 Asignación[('C4', 'I4')] + 60.0 Asignación[('C5', 'I1')] + 119.0 Asignación[('C5', 'I3')] + 114.0 Asignación[('C5', 'I5')] + 64.0 Asignación[('C5', 'I6')] + 113.0 Asignación[('C5', 'I2')] + 112.0 Asignación[('C5', 'I4')] 

Constraints:
  Asignación[('C2', 'I1')] + Asignación[('C5', 'I1')] <= 2.0
  Asignación[('C2', 'I2')] + Asignación[('C4', 'I2')] + Asignación[('C5', 'I2')] <= 1.0
  Asignación[('C1', 'I3')] + Asignación[('C3', 'I3')] + Asignación[('C5', 'I3')] <= 3.0
  Asignación[('C3', 'I4')] + Asignación[('C4', 'I4')] + Asignación[('C5', 'I4')] <= 2.0
  Asignación[('C5', 'I5')] <= 1.0
  Asignación[('C1', 'I6')] + Asignación[('C2', 'I6')] + Asignación[('C3', 'I6')] + Asigna

<br/>

En la siguiente sección de código se le indica a la herramienta "Sagemath" que resuelva el modelo de programación lineal con la función objetivo y restricciones establecidas.
Para este caso en particular, el costo mínimo de asignación de ingenieros a solicitudes de clientes es de 745.00 dólares.    
Este es el costo mínimo que se puede obtener bajo las restricciones especificadas en el modelo.

In [10]:
solucion = programa.solve()
print(solucion)

745.0


<br/>

En esta última parte del código, se muestra la solución desglosada para la cantidad de servicios por cliente y qué ingeniero atiende cada uno de ellos, además se indica el costo de atención de cada solicitud.

In [11]:
suma = 0
previouskey = ''
for key, val in programa.get_values(asignacion).items():
    ##if val != 0:
    solicitudestotal = solicitudes.Servicio01.loc[key[0]] + solicitudes.Servicio02.loc[key[0]] + solicitudes.Servicio03.loc[key[0]] + solicitudes.Servicio04.loc[key[0]] + solicitudes.Servicio05.loc[key[0]] + solicitudes.Servicio06.loc[key[0]]
    if previouskey != key[0]:
        print("El cliente", key[0] , "solicitó", solicitudestotal, "servicios")
        previouskey = key[0]
    parcial = val * costos[key[0]].loc[key[1]]
    suma += parcial
    print("\tEl ingeniero", key[1] , "dara {:.0f}".format(val), "servicios a costo de ${:,.2f}".format(parcial))
print("\nEl costo del programa es ${:,.2f}".format(suma))

El cliente C1 solicitó 2 servicios
	El ingeniero I3 dara 2 servicios a costo de $222.00
	El ingeniero I6 dara 0 servicios a costo de $0.00
El cliente C2 solicitó 1 servicios
	El ingeniero I1 dara 0 servicios a costo de $0.00
	El ingeniero I2 dara 1 servicios a costo de $97.00
	El ingeniero I6 dara 0 servicios a costo de $0.00
El cliente C3 solicitó 2 servicios
	El ingeniero I3 dara 0 servicios a costo de $0.00
	El ingeniero I6 dara 2 servicios a costo de $120.00
	El ingeniero I4 dara 0 servicios a costo de $0.00
El cliente C4 solicitó 2 servicios
	El ingeniero I2 dara 0 servicios a costo de $0.00
	El ingeniero I4 dara 2 servicios a costo de $186.00
El cliente C5 solicitó 2 servicios
	El ingeniero I1 dara 2 servicios a costo de $120.00
	El ingeniero I3 dara 0 servicios a costo de $0.00
	El ingeniero I5 dara 0 servicios a costo de $0.00
	El ingeniero I6 dara 0 servicios a costo de $0.00
	El ingeniero I2 dara 0 servicios a costo de $0.00
	El ingeniero I4 dara 0 servicios a costo de $0.00
